<a href="https://colab.research.google.com/github/AkilaAgnesS/lung-nodule-detection/blob/main/NoduleSegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This completes the implementation with:

The full CTDataset class for handling CT images and masks
Training and evaluation functions
Helper functions for calculating the Dice coefficient (a common metric for segmentation)
Functions for model saving/loading
Visualization utilities
An example of how to use the code with sample data

The implementation showcases how to use a Swin Transformer-based UNet for CT image segmentation with edge detection enhancement via the Canny algorithm and feature pyramid network (FPN) for feature aggregation.

In [ ]:
# Swin Transformer UNet for CT Image Segmentation
# Google Colab Implementation

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import os
from google.colab import drive
import warnings
warnings.filterwarnings('ignore')

# Mount Google Drive for saving models and results
drive.mount('/content/drive', force_remount=True)

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import os
from sklearn.model_selection import train_test_split
import math
import copy
from skimage.transform import resize
from matplotlib.patches import Rectangle
import matplotlib.patheffects as path_effects
from skimage.metrics import normalized_mutual_information
try:
    from skimage.metrics import structural_similarity
except:
    from skimage.measure import compare_ssim as structural_similarity

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Dataset class for loading .npy files
class LungCTDataset(Dataset):
    def __init__(self, image_paths, mask_paths=None, transform=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = np.load(self.image_paths[idx])

        # Resize to 512x512 if needed
        if image.shape[0] != 512 or image.shape[1] != 512:
            image = resize(image, (512, 512), preserve_range=True)

        # Convert to float tensors and normalize image to [0, 1]
        image = torch.tensor(image, dtype=torch.float32)
        if len(image.shape) == 2:  # Add channel dimension if needed
            image = image.unsqueeze(0)

        # Handle extreme values and NaNs
        image = torch.nan_to_num(image)
        if torch.max(image) > 0:
            image = image / torch.max(image)  # Normalize to [0, 1]

        # Handle mask if available
        if self.mask_paths is not None:
            mask = np.load(self.mask_paths[idx])

            # Resize if needed
            if mask.shape[0] != 512 or mask.shape[1] != 512:
                mask = resize(mask, (512, 512), preserve_range=True)

            mask = torch.tensor(mask, dtype=torch.float32)
            if len(mask.shape) == 2:  # Add channel dimension
                mask = mask.unsqueeze(0)

            # Convert mask to binary (0 or 1)
            mask = (mask > 0.5).float()
        else:
            # If no mask is provided, return a dummy mask
            mask = torch.zeros_like(image)

        if self.transform:
            image = self.transform(image)

        return image, mask

# Utility function to create patches from an image
def img_to_patch(x, patch_size, flatten_channels=True):
    """
    Args:
        x: Input image tensor of shape [B, C, H, W]
        patch_size: Size of patches to extract
        flatten_channels: If True, channels are flattened with patch dim
    Returns:
        Tensor of shape [B, num_patches, C*patch_size*patch_size] if flatten_channels=True
        else [B, num_patches, C, patch_size, patch_size]
    """
    B, C, H, W = x.shape
    x = x.reshape(B, C, H//patch_size, patch_size, W//patch_size, patch_size)
    x = x.permute(0, 2, 4, 1, 3, 5)  # [B, H', W', C, p, p]
    x = x.flatten(1, 2)  # [B, H'*W', C, p, p]
    if flatten_channels:
        x = x.flatten(2, 4)  # [B, H'*W', C*p*p]
    return x

# Patch Embedding layer
class PatchEmbedding(nn.Module):
    def __init__(self, in_channels, emb_dim, patch_size, img_size):
        super().__init__()
        self.patch_size = patch_size
        num_patches = (img_size // patch_size) ** 2
        self.projection = nn.Conv2d(in_channels, emb_dim, kernel_size=patch_size, stride=patch_size)
        self.norm = nn.LayerNorm(emb_dim)

    def forward(self, x):
        """x: [B, C, H, W]"""
        B, C, H, W = x.shape
        x = self.projection(x)  # [B, emb_dim, H/patch_size, W/patch_size]
        x = x.flatten(2).transpose(1, 2)  # [B, num_patches, emb_dim]
        x = self.norm(x)
        return x

# Window Attention Module
class WindowAttention(nn.Module):
    def __init__(self, dim, window_size, num_heads):
        super().__init__()
        self.dim = dim
        self.window_size = window_size
        self.num_heads = num_heads
        self.head_dim = dim // num_heads
        self.scale = self.head_dim ** -0.5

        # Linear projection layers
        self.qkv = nn.Linear(dim, dim * 3, bias=True)
        self.proj = nn.Linear(dim, dim)

        # Relative position bias
        self.relative_position_bias_table = nn.Parameter(
            torch.zeros((2 * window_size - 1) * (2 * window_size - 1), num_heads)
        )

        # Pair-wise relative position indices
        coords_h = torch.arange(self.window_size)
        coords_w = torch.arange(self.window_size)
        coords = torch.stack(torch.meshgrid([coords_h, coords_w], indexing="ij"))
        coords_flatten = torch.flatten(coords, 1)
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]
        relative_coords = relative_coords.permute(1, 2, 0).contiguous()
        relative_coords[:, :, 0] += self.window_size - 1  # shift to start from 0
        relative_coords[:, :, 1] += self.window_size - 1
        relative_coords[:, :, 0] *= 2 * self.window_size - 1
        relative_position_index = relative_coords.sum(-1)
        self.register_buffer("relative_position_index", relative_position_index)

        nn.init.trunc_normal_(self.relative_position_bias_table, std=0.02)

    def forward(self, x):
        """x: [num_windows*B, window_size*window_size, dim]"""
        B_, N, C = x.shape

        # Generate Q, K, V
        qkv = self.qkv(x).reshape(B_, N, 3, self.num_heads, self.head_dim).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]  # each [B_, num_heads, N, head_dim]

        # Compute attention
        q = q * self.scale
        attn = (q @ k.transpose(-2, -1))  # [B_, num_heads, N, N]

        # Add relative position bias
        relative_position_bias = self.relative_position_bias_table[self.relative_position_index.view(-1)].view(
            self.window_size * self.window_size, self.window_size * self.window_size, -1
        )
        relative_position_bias = relative_position_bias.permute(2, 0, 1).contiguous()  # [num_heads, N, N]
        attn = attn + relative_position_bias.unsqueeze(0)

        attn = F.softmax(attn, dim=-1)

        # Weighted aggregation
        x = (attn @ v).transpose(1, 2).reshape(B_, N, C)
        x = self.proj(x)

        return x

# Window partitioning function
def window_partition(x, window_size):
    """
    Args:
        x: [B, H, W, C]
        window_size: window size
    Returns:
        windows: [num_windows*B, window_size, window_size, C]
    """
    B, H, W, C = x.shape

    # Ensure dimensions are divisible by window_size
    pad_h = (window_size - H % window_size) % window_size
    pad_w = (window_size - W % window_size) % window_size

    if pad_h > 0 or pad_w > 0:
        x = F.pad(x, (0, 0, 0, pad_w, 0, pad_h))
        H, W = H + pad_h, W + pad_w

    x = x.view(B, H // window_size, window_size, W // window_size, window_size, C)
    windows = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(-1, window_size, window_size, C)
    return windows

# Window reverse function
def window_reverse(windows, window_size, H, W):
    """
    Args:
        windows: [num_windows*B, window_size, window_size, C]
        window_size: Window size
        H, W: Height and width of the image
    Returns:
        x: [B, H, W, C]
    """
    # Ensure H and W are divisible by window_size
    H_orig, W_orig = H, W
    pad_h = (window_size - H % window_size) % window_size
    pad_w = (window_size - W % window_size) % window_size
    H, W = H + pad_h, W + pad_w

    B = int(windows.shape[0] / (H * W / window_size / window_size))
    x = windows.view(B, H // window_size, W // window_size, window_size, window_size, -1)
    x = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(B, H, W, -1)

    # Remove padding if added
    if pad_h > 0 or pad_w > 0:
        x = x[:, :H_orig, :W_orig, :]

    return x

# MLP Block
class MLP(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

# Swin Transformer Block
class SwinTransformerBlock(nn.Module):
    def __init__(self, dim, num_heads, window_size=7, shift_size=0, mlp_ratio=4., drop=0.):
        super().__init__()
        self.dim = dim
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift_size = shift_size
        self.mlp_ratio = mlp_ratio

        # Layer Normalization
        self.norm1 = nn.LayerNorm(dim)
        self.norm2 = nn.LayerNorm(dim)

        # Window Attention
        self.attn = WindowAttention(dim, window_size, num_heads)

        # MLP
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = MLP(in_features=dim, hidden_features=mlp_hidden_dim, drop=drop)

        # Handling boundaries when shifting windows
        self.H = None
        self.W = None

    def forward(self, x, mask_matrix=None):
        """x: [B, L, C]"""
        B, L, C = x.shape

        # Determine H and W ensuring they're square to avoid dimension issues
        H = W = int(math.sqrt(L))

        # If L is not a perfect square, we need to pad
        if H * W != L:
            nearest_sq = int(math.ceil(math.sqrt(L)))
            H = W = nearest_sq
            # Pad the sequence to make it a perfect square
            padding = H * W - L
            if padding > 0:
                padding_tensor = torch.zeros(B, padding, C, device=x.device, dtype=x.dtype)
                x = torch.cat([x, padding_tensor], dim=1)
                L = H * W

        x = x.view(B, H, W, C)

        # Cache size for later use
        self.H, self.W = H, W

        # Skip connection
        shortcut = x
        x = self.norm1(x)  # [B, H, W, C]

        # Cyclic shift
        if self.shift_size > 0:
            shifted_x = torch.roll(x, shifts=(-self.shift_size, -self.shift_size), dims=(1, 2))
        else:
            shifted_x = x

        # Partition windows
        x_windows = window_partition(shifted_x, self.window_size)  # [num_windows*B, window_size, window_size, C]
        x_windows = x_windows.view(-1, self.window_size * self.window_size, C)  # [num_windows*B, window_size*window_size, C]

        # W-MSA/SW-MSA
        attn_windows = self.attn(x_windows)  # [num_windows*B, window_size*window_size, C]

        # Merge windows
        attn_windows = attn_windows.view(-1, self.window_size, self.window_size, C)
        shifted_x = window_reverse(attn_windows, self.window_size, H, W)  # [B, H, W, C]

        # Reverse cyclic shift
        if self.shift_size > 0:
            x = torch.roll(shifted_x, shifts=(self.shift_size, self.shift_size), dims=(1, 2))
        else:
            x = shifted_x

        # Reshape back to [B, L, C]
        x = x.view(B, H * W, C)

        # If we padded earlier, remove the padding
        if H * W > L:
            x = x[:, :L, :]

        # FFN
        x = shortcut.view(B, -1, C) + x
        x = x + self.mlp(self.norm2(x))

        return x

# Patch Merging layer
class PatchMerging(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.reduction = nn.Linear(4 * in_dim, out_dim, bias=False)
        self.norm = nn.LayerNorm(4 * in_dim)

    def forward(self, x):
        """x: [B, H*W, C]"""
        B, L, C = x.shape
        H = W = int(math.sqrt(L))

        # Reshape to [B, H, W, C]
        x = x.view(B, H, W, C)

        # Concatenate 2x2 patches
        x0 = x[:, 0::2, 0::2, :]  # [B, H/2, W/2, C]
        x1 = x[:, 1::2, 0::2, :]  # [B, H/2, W/2, C]
        x2 = x[:, 0::2, 1::2, :]  # [B, H/2, W/2, C]
        x3 = x[:, 1::2, 1::2, :]  # [B, H/2, W/2, C]
        x = torch.cat([x0, x1, x2, x3], dim=-1)  # [B, H/2, W/2, 4*C]
        x = x.view(B, -1, 4 * C)  # [B, H/2*W/2, 4*C]

        # Linear projection
        x = self.norm(x)
        x = self.reduction(x)

        return x

# Patch Expanding layer
class PatchExpanding(nn.Module):
    def __init__(self, in_dim, out_dim, scale_factor=2):
        super().__init__()
        self.scale_factor = scale_factor
        self.expansion = nn.Linear(in_dim, scale_factor * scale_factor * out_dim, bias=False)

    def forward(self, x):
        """x: [B, H*W, C]"""
        B, L, C = x.shape
        H = W = int(math.sqrt(L))

        # Expand features
        x = self.expansion(x)  # [B, H*W, scale_factor*scale_factor*out_dim]

        # Reshape and rearrange to increase spatial size
        x = x.view(B, H, W, self.scale_factor, self.scale_factor, -1)
        x = x.permute(0, 1, 3, 2, 4, 5).contiguous()
        out_dim = x.shape[-1]
        x = x.view(B, H * self.scale_factor, W * self.scale_factor, out_dim)
        x = x.view(B, -1, out_dim)  # [B, (H*scale_factor)*(W*scale_factor), out_dim]

        return x

# Basic Swin Transformer Layer (multiple blocks with skip connection)
class BasicLayer(nn.Module):
    def __init__(self, dim, depth, num_heads, window_size, mlp_ratio=4.):
        super().__init__()
        self.dim = dim
        self.depth = depth

        # Build Swin Transformer blocks
        self.blocks = nn.ModuleList([
            SwinTransformerBlock(
                dim=dim,
                num_heads=num_heads,
                window_size=window_size,
                shift_size=0 if (i % 2 == 0) else window_size // 2,
                mlp_ratio=mlp_ratio
            )
            for i in range(depth)
        ])

    def forward(self, x):
        for block in self.blocks:
            x = block(x)
        return x

# Encoder part of Swin Transformer U-Net
class SwinTransformerEncoder(nn.Module):
    def __init__(self, in_channels, depths=[2, 2, 2], embed_dims=[96, 192, 384], num_heads=[3, 6, 12], window_size=7):
        super().__init__()
        self.depths = depths
        self.num_stages = len(depths)

        # Initial patch embedding - use patch_size=8 for 512x512 images
        self.patch_embed = PatchEmbedding(in_channels, embed_dims[0], patch_size=8, img_size=512)

        # Swin Transformer layers
        self.layers = nn.ModuleList()
        for i in range(self.num_stages):
            layer = BasicLayer(
                dim=embed_dims[i],
                depth=depths[i],
                num_heads=num_heads[i],
                window_size=window_size
            )
            self.layers.append(layer)

            # Add patch merging except for the last layer
            if i < self.num_stages - 1:
                self.layers.append(PatchMerging(embed_dims[i], embed_dims[i+1]))

    def forward(self, x):
        x = self.patch_embed(x)  # [B, L, C]

        features = []
        for i, layer in enumerate(self.layers):
            x = layer(x)
            if isinstance(layer, BasicLayer):
                features.append(x)

        return features

# Decoder part of Swin Transformer U-Net
class SwinTransformerDecoder(nn.Module):
    def __init__(self, depths=[2, 2, 2], embed_dims=[384, 192, 96], num_heads=[12, 6, 3], window_size=7):
        super().__init__()
        self.depths = depths
        self.num_stages = len(depths)

        # Layers for decoding (in reverse order)
        self.layers = nn.ModuleList()
        for i in range(self.num_stages):
            # Patch Expanding
            self.layers.append(PatchExpanding(embed_dims[i], embed_dims[min(i+1, len(embed_dims)-1)]))

            # Swin Transformer Block
            self.layers.append(BasicLayer(
                dim=embed_dims[min(i+1, len(embed_dims)-1)],
                depth=depths[i],
                num_heads=num_heads[i],
                window_size=window_size
            ))

    def forward(self, features):
        # Start from the deepest feature map
        x = features[-1]

        # Process through layers with skip connections
        for i in range(0, len(self.layers), 2):
            # Expanding
            x = self.layers[i](x)

            # Skip connection (if available)
            if i//2 < len(features) - 1:
                x = x + features[len(features) - 2 - i//2]

            # Swin transformer blocks
            x = self.layers[i+1](x)

        return x

# Feature Pyramid Network (FPN)
class FPN(nn.Module):
    def __init__(self, in_channels_list, out_channels):
        super().__init__()

        # Lateral connections
        self.lateral_convs = nn.ModuleList([
            nn.Linear(in_channels, out_channels)
            for in_channels in in_channels_list
        ])

        # Output convolutions
        self.output_convs = nn.ModuleList([
            nn.Sequential(
                nn.Linear(out_channels, out_channels),
                nn.LayerNorm(out_channels),
                nn.GELU()
            )
            for _ in in_channels_list
        ])

    def forward(self, features):
        """features: List of feature maps from different levels"""

        # Process each feature map
        laterals = [conv(feature) for feature, conv in zip(features, self.lateral_convs)]

        # Top-down pathway
        for i in range(len(laterals)-1, 0, -1):
            # Ensure spatial dimensions match
            B, L_high, C = laterals[i].shape
            B, L_low, C = laterals[i-1].shape

            H_high = W_high = int(math.sqrt(L_high))
            H_low = W_low = int(math.sqrt(L_low))

            # Reshape for interpolation
            laterals[i] = laterals[i].view(B, H_high, W_high, C)
            laterals[i] = F.interpolate(
                laterals[i].permute(0, 3, 1, 2),
                size=(H_low, W_low),
                mode='bilinear',
                align_corners=False
            ).permute(0, 2, 3, 1).contiguous().view(B, L_low, C)

            # Feature fusion
            laterals[i-1] = laterals[i-1] + laterals[i]

        # Apply output convolutions
        outs = [output_conv(lateral) for lateral, output_conv in zip(laterals, self.output_convs)]

        return outs

# Complete Swin Transformer Segmentation Model
class SwinTransformerSegmentation(nn.Module):
    def __init__(self, in_channels=1, out_channels=1, depths=[2, 2, 2],
                 embed_dims=[96, 192, 384], num_heads=[3, 6, 12], window_size=7):
        super().__init__()

        # Encoder
        self.encoder = SwinTransformerEncoder(
            in_channels=in_channels,
            depths=depths,
            embed_dims=embed_dims,
            num_heads=num_heads,
            window_size=window_size
        )

        # Decoder
        self.decoder = SwinTransformerDecoder(
            depths=depths[::-1],
            embed_dims=embed_dims[::-1],
            num_heads=num_heads[::-1],
            window_size=window_size
        )

        # Feature Pyramid Network
        self.fpn = FPN(embed_dims, embed_dims[0])

        # Edge detection branch
        self.edge_detection = nn.Sequential(
            nn.Conv2d(in_channels, 16, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(16, 1, kernel_size=1)
        )

        # Final prediction layers
        self.final_conv = nn.Sequential(
            nn.Linear(embed_dims[0], 64),
            nn.GELU(),
            nn.Linear(64, out_channels)
        )

    def forward(self, x):
        # Edge detection branch
        edge_features = self.edge_detection(x)

        # Encoder
        encoder_features = self.encoder(x)

        # Decoder
        decoder_output = self.decoder(encoder_features)

        # FPN for feature aggregation
        fpn_features = self.fpn([encoder_features[0], encoder_features[1], encoder_features[2]])

        # Final prediction
        B, L, C = decoder_output.shape
        H = W = int(math.sqrt(L))

        # Reshape to spatial dimensions
        out = self.final_conv(decoder_output)  # [B, L, out_channels]
        out = out.view(B, H, W, -1).permute(0, 3, 1, 2)

        # Upsample to input resolution
        out = F.interpolate(out, size=(512, 512), mode='bilinear', align_corners=False)

        # Combine with edge features
        out = out + edge_features

        return torch.sigmoid(out)

# Training function
def train_model(model, train_loader, val_loader, num_epochs=10, learning_rate=1e-4):
    # Loss function and optimizer
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Learning rate scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5, verbose=True
    )

    # Training loop
    best_val_loss = float('inf')
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0

        for images, masks in train_loader:
            images = images.to(device)
            masks = masks.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, masks)

            # Backward pass and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Calculate average loss for the epoch
        train_loss /= len(train_loader)

        # Validation phase
        model.eval()
        val_loss = 0.0
        dice_score = 0.0

        with torch.no_grad():
            for images, masks in val_loader:
                images = images.to(device)
                masks = masks.to(device)

                outputs = model(images)
                loss = criterion(outputs, masks)
                val_loss += loss.item()

                # Calculate Dice coefficient
                pred = (outputs > 0.5).float()
                intersection = (pred * masks).sum().item()
                dice = (2 * intersection) / (pred.sum().item() + masks.sum().item() + 1e-8)
                dice_score += dice

        # Calculate average validation loss and Dice score
        val_loss /= len(val_loader)
        dice_score /= len(val_loader)

        # Update learning rate based on validation loss
        scheduler.step(val_loss)

        # Print progress
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Dice: {dice_score:.4f}')

        # Save the best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pth')
            print(f'Model saved at epoch {epoch+1}')

    return model

# Inference function with DSC score overlay
def predict(model, image_path, mask_path=None, output_path=None):
    """
    Run inference and create visualization with DSC score overlay if mask is available
    """
    import numpy as np
    import matplotlib.pyplot as plt
    from matplotlib.patches import Rectangle
    import matplotlib.patheffects as path_effects

    # Load image
    image = np.load(image_path)

    # Preprocess
    image_tensor = torch.tensor(image, dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # Add batch and channel dims
    image_tensor = image_tensor / torch.max(image_tensor)  # Normalize

    # Predict
    model.eval()
    with torch.no_grad():
        image_tensor = image_tensor.to(device)
        output = model(image_tensor)
        prediction = (output > 0.5).float().cpu().numpy().squeeze()

    # Calculate evaluation metrics if mask is available
    dsc_score = None
    if mask_path:
        try:
            # Load mask
            true_mask = np.load(mask_path)

            # Ensure it's binary
            true_mask = (true_mask > 0.5).astype(np.float32)

            # Calculate Dice coefficient
            smooth = 1e-6
            intersection = np.sum(prediction * true_mask)
            dsc_score = (2. * intersection + smooth) / (np.sum(prediction) + np.sum(true_mask) + smooth)

            # Calculate other metrics
            nmi_score = normalized_mutual_information(true_mask, prediction)
            try:
                ssim_score = structural_similarity(true_mask, prediction)
            except:
                ssim_score = 0.0

            print(f"Metrics: DSC={dsc_score:.4f}, NMI={nmi_score:.4f}, SSIM={ssim_score:.4f}")
        except Exception as e:
            print(f"Error calculating metrics: {str(e)}")

    # Create visualization similar to the example
    plt.figure(figsize=(10, 8))

    # Show the original CT image with prediction overlay
    plt.imshow(image, cmap='gray')

    # Add segmentation overlay with transparency
    mask_overlay = np.zeros((*prediction.shape, 4))
    mask_overlay[prediction > 0.5] = [1, 0, 0, 0.3]  # Red with transparency
    plt.imshow(mask_overlay)

    # Add DSC score in a box if available
    if dsc_score is not None:
        # Add box in top-left corner
        ax = plt.gca()
        box = Rectangle((5, 5), 100, 30, facecolor='black', alpha=0.7)
        ax.add_patch(box)

        # Add DSC score text
        text = plt.text(10, 25, f"DSC: {dsc_score:.2f}", color='white', fontsize=12, weight='bold')

        # Add outline to make text more readable
        text.set_path_effects([
            path_effects.Stroke(linewidth=2, foreground='black'),
            path_effects.Normal()
        ])

    plt.axis('off')
    plt.tight_layout()

    # Save if output path is provided
    if output_path:
        plt.savefig(output_path, dpi=200, bbox_inches='tight')
        print(f"Saved visualization to {output_path}")

    plt.show()

    return prediction

In [ ]:
# Enhanced lung nodule detection model with focused training
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from skimage.transform import resize
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib.patheffects as path_effects
from sklearn.model_selection import train_test_split

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

###### STEP 1: LOAD AND PREPARE THE DATA ######

# Path to your .npy files
image_path = '/content/drive/My Drive/Lung/2D_img_512.npy'
mask_path = '/content/drive/My Drive/Lung/2D_mask_512.npy'

# Load the data
images = np.load(image_path)
masks = np.load(mask_path)
print(f"Loaded {images.shape[0]} images with shape {images.shape[1:]}")
print(f"Loaded {masks.shape[0]} masks with shape {masks.shape[1:]}")

# Find images with significant nodules (where mask has substantial positive area)
nodule_indices = []
for i in range(len(masks)):
    mask_area = np.sum(masks[i] > 0.5)
    if mask_area > 100:  # Threshold to ensure the nodule is substantial
        nodule_indices.append(i)

print(f"Found {len(nodule_indices)} images with significant nodules")

# If we have very few images with nodules, adjust the threshold
if len(nodule_indices) < 10:
    nodule_indices = []
    for i in range(len(masks)):
        mask_area = np.sum(masks[i] > 0.5)
        if mask_area > 0:  # Any nodule
            nodule_indices.append(i)
    print(f"Adjusted threshold: Found {len(nodule_indices)} images with any nodules")

# Select a sample with a clear nodule for later inference
if nodule_indices:
    # Find the image with the largest nodule
    max_nodule_idx = max(nodule_indices, key=lambda i: np.sum(masks[i] > 0.5))
    sample_idx = max_nodule_idx
else:
    sample_idx = 100  # Default

sample_image = images[sample_idx].copy()
sample_mask = masks[sample_idx].copy()

print(f"Selected sample index {sample_idx} with nodule area: {np.sum(sample_mask > 0.5)} pixels")

# Visualize the sample image and mask
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(sample_image, cmap='gray')
plt.title(f'Sample Image (Index: {sample_idx})')
plt.subplot(1, 2, 2)
plt.imshow(sample_mask, cmap='gray')
plt.title('Nodule Mask')
plt.tight_layout()
plt.show()

# Prepare dataset for training with class weighting for better nodule detection
class LungNoduleDataset(Dataset):
    def __init__(self, images, masks, transform=None):
        self.images = images
        self.masks = masks
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        mask = self.masks[idx]

        # Normalize image to [0,1]
        if np.max(image) > 0:
            image = image / np.max(image)

        # Ensure mask is binary
        mask = (mask > 0.5).astype(np.float32)

        # Convert to tensors
        image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)  # Add channel dimension
        mask = torch.tensor(mask, dtype=torch.float32).unsqueeze(0)    # Add channel dimension

        return image, mask

# Use nodule indices to ensure we train primarily on images with nodules
if nodule_indices:
    # Split nodule indices into training and validation
    train_nodule_indices, val_nodule_indices = train_test_split(
        nodule_indices, test_size=0.2, random_state=42
    )

    # Add some non-nodule images for balanced training
    non_nodule_indices = [i for i in range(len(images)) if i not in nodule_indices]
    if non_nodule_indices:
        # Add equal number of non-nodule images
        num_non_nodules = min(len(train_nodule_indices), len(non_nodule_indices))
        selected_non_nodules = np.random.choice(non_nodule_indices, num_non_nodules, replace=False)
        train_indices = list(train_nodule_indices) + list(selected_non_nodules)

        # Add some non-nodule images to validation
        num_val_non_nodules = min(len(val_nodule_indices), len(non_nodule_indices))
        val_non_nodule_indices = np.random.choice(
            [i for i in non_nodule_indices if i not in selected_non_nodules],
            num_val_non_nodules, replace=False
        )
        val_indices = list(val_nodule_indices) + list(val_non_nodule_indices)
    else:
        train_indices = train_nodule_indices
        val_indices = val_nodule_indices
else:
    # If no clear nodules, use regular train/val split
    train_indices, val_indices = train_test_split(
        range(len(images)), test_size=0.2, random_state=42
    )

print(f"Training on {len(train_indices)} images ({len([i for i in train_indices if i in nodule_indices])} with nodules)")
print(f"Validating on {len(val_indices)} images ({len([i for i in val_indices if i in nodule_indices])} with nodules)")

# Create the actual training and validation datasets
train_images = np.array([images[i] for i in train_indices])
train_masks = np.array([masks[i] for i in train_indices])
val_images = np.array([images[i] for i in val_indices])
val_masks = np.array([masks[i] for i in val_indices])

# Create datasets
train_dataset = LungNoduleDataset(train_images, train_masks)
val_dataset = LungNoduleDataset(val_images, val_masks)

# Create data loaders
batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

###### STEP 2: DEFINE A BETTER MODEL FOR NODULE DETECTION ######

# U-Net model for better nodule segmentation
# U-Net model for better nodule segmentation - Fixed version
class UNet(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super(UNet, self).__init__()

        # Encoder
        self.enc1 = self._make_encoder_block(in_channels, 64)
        self.enc2 = self._make_encoder_block(64, 128)
        self.enc3 = self._make_encoder_block(128, 256)
        self.enc4 = self._make_encoder_block(256, 512)

        # Bridge
        self.bridge = self._make_encoder_block(512, 1024)

        # Decoder
        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec4 = self._make_decoder_block(1024, 512)  # 1024 = 512 + 512 (skip)

        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = self._make_decoder_block(512, 256)   # 512 = 256 + 256 (skip)

        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = self._make_decoder_block(256, 128)   # 256 = 128 + 128 (skip)

        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = self._make_decoder_block(128, 64)    # 128 = 64 + 64 (skip)

        # Final layer
        self.final = nn.Conv2d(64, out_channels, kernel_size=1)

        # Max pooling for encoder
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def _make_encoder_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def _make_decoder_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        # Encoder
        enc1 = self.enc1(x)

        x = self.pool(enc1)
        enc2 = self.enc2(x)

        x = self.pool(enc2)
        enc3 = self.enc3(x)

        x = self.pool(enc3)
        enc4 = self.enc4(x)

        # Bridge
        x = self.pool(enc4)
        bridge = self.bridge(x)

        # Decoder with skip connections
        x = self.upconv4(bridge)
        x = torch.cat([x, enc4], dim=1)
        x = self.dec4(x)

        x = self.upconv3(x)
        x = torch.cat([x, enc3], dim=1)
        x = self.dec3(x)

        x = self.upconv2(x)
        x = torch.cat([x, enc2], dim=1)
        x = self.dec2(x)

        x = self.upconv1(x)
        x = torch.cat([x, enc1], dim=1)
        x = self.dec1(x)

        # Final output
        return torch.sigmoid(self.final(x))

# Initialize and train the model
model = UNet(in_channels=1, out_channels=1).to(device)
model = train_model(model, train_loader, val_loader, num_epochs=15)  # More epochs for better training

###### STEP 4: RUN INFERENCE WITH IMPROVED VISUALIZATION ######

def predict_with_dsc_overlay(model, image, mask=None):
    """Run inference and create visualization with DSC score overlay"""
    # Prepare image for the model
    image_tensor = torch.tensor(image, dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # Add batch and channel dims
    if np.max(image) > 0:
        image_tensor = image_tensor / torch.max(image_tensor)  # Normalize

    # Predict
    model.eval()
    with torch.no_grad():
        image_tensor = image_tensor.to(device)
        output = model(image_tensor)
        prediction_prob = output.cpu().numpy().squeeze()  # Keep probabilities
        prediction = (prediction_prob > 0.5).astype(np.float32)  # Binary prediction

    # Calculate evaluation metrics if mask is provided
    dsc_score = None
    if mask is not None:
        # Ensure mask is binary
        true_mask = (mask > 0.5).astype(np.float32)

        # Calculate Dice coefficient
        smooth = 1e-6
        intersection = np.sum(prediction * true_mask)
        dsc_score = (2. * intersection + smooth) / (np.sum(prediction) + np.sum(true_mask) + smooth)
        print(f"DSC Score: {dsc_score:.4f}")

    # Create visualization
    plt.figure(figsize=(10, 8))

    # Show the original CT image
    plt.imshow(image, cmap='gray')

    # Add segmentation overlay with transparency
    mask_overlay = np.zeros((*prediction.shape, 4))
    mask_overlay[prediction > 0.5] = [1, 0, 0, 0.3]  # Red with transparency
    plt.imshow(mask_overlay)

    # Add DSC score in a box if available
    if dsc_score is not None:
        # Add box in top-left corner
        ax = plt.gca()
        box = Rectangle((5, 5), 100, 30, facecolor='black', alpha=0.7)
        ax.add_patch(box)

        # Add DSC score text
        text = plt.text(10, 25, f"DSC: {dsc_score:.2f}", color='white', fontsize=12, weight='bold')

        # Add outline to make text more readable
        text.set_path_effects([
            path_effects.Stroke(linewidth=2, foreground='black'),
            path_effects.Normal()
        ])

    plt.axis('off')
    plt.tight_layout()

    # Save result
    plt.savefig('/content/sample_prediction.png', dpi=200, bbox_inches='tight')
    plt.show()

    # Create a detailed comparison figure with heatmap
    plt.figure(figsize=(20, 5))

    # Original image
    plt.subplot(1, 4, 1)
    plt.imshow(image, cmap='gray')
    plt.title('Original Image')
    plt.axis('off')

    # Ground truth mask
    plt.subplot(1, 4, 2)
    plt.imshow(mask, cmap='gray')
    plt.title('Ground Truth')
    plt.axis('off')

    # Model prediction (binary)
    plt.subplot(1, 4, 3)
    plt.imshow(prediction, cmap='gray')
    plt.title(f'Prediction (DSC: {dsc_score:.2f})')
    plt.axis('off')

    # Probability heatmap
    plt.subplot(1, 4, 4)
    plt.imshow(prediction_prob, cmap='hot')
    plt.colorbar(label='Probability')
    plt.title('Probability Heatmap')
    plt.axis('off')

    plt.tight_layout()
    plt.savefig('/content/detailed_comparison.png', dpi=200, bbox_inches='tight')
    plt.show()

    return prediction

# Run inference on the sample image
print("\n--- Running inference on sample image with nodule ---")
prediction = predict_with_dsc_overlay(model, sample_image, sample_mask)

# Save the trained model
torch.save(model.state_dict(), '/content/lung_nodule_detection_model.pth')
print("Model saved to /content/lung_nodule_detection_model.pth")
print("Sample prediction saved to /content/sample_prediction.png")
print("Detailed comparison saved to /content/detailed_comparison.png")